In [9]:
import psycopg2
import os
import glob
import json

In [2]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  is_local = False
except ModuleNotFoundError:
  is_local = True

In [3]:
folder_landing = "./landing" if (is_local) else "/content/drive/MyDrive/ADSDB/landing"

folder_temporal = os.path.join(folder_landing, "temporal")
folder_persistent = os.path.join(folder_landing, "persistent")

extract_dir = os.path.join(folder_persistent, "extracted")

In [4]:
table_spec = dict(
    MortICD = '''
    "Country" integer,
    "Admin1" VARCHAR (3),
    "SubDiv" VARCHAR (3),
    "Year" integer,
    "List" VARCHAR (3),
    "Cause" VARCHAR (4),
    "Sex" integer,
    "Frmat" VARCHAR (2),
    "IM_Frmat" VARCHAR (2),
    "Deaths1" integer,
    "Deaths2" integer,
    "Deaths3" integer,
    "Deaths4" integer,
    "Deaths5" integer,
    "Deaths6" integer,
    "Deaths7" integer,
    "Deaths8" integer,
    "Deaths9" integer,
    "Deaths10" integer,
    "Deaths11" integer,
    "Deaths12" integer,
    "Deaths13" integer,
    "Deaths14" integer,
    "Deaths15" integer,
    "Deaths16" integer,
    "Deaths17" integer,
    "Deaths18" integer,
    "Deaths19" integer,
    "Deaths20" integer,
    "Deaths21" integer,
    "Deaths22" integer,
    "Deaths23" integer,
    "Deaths24" integer,
    "Deaths25" integer,
    "Deaths26" integer,
    "IM_Deaths1" integer,
    "IM_Deaths2" integer,
    "IM_Deaths3" integer,
    "IM_Deaths4" integer
    ''',
    CountryCodes = '''
    "Country" integer PRIMARY KEY,
    "Name" VARCHAR(50)
    ''',
    Notes = '''
    "Country" integer,
    "Year" integer,
    "Note" VARCHAR (100)
    ''',
    Population = '''
    "Country" integer,
    "Admin1" VARCHAR (3),
    "SubDiv" VARCHAR (3),
    "Year" integer,
    "Sex" integer,
    "Frmat" VARCHAR (2),
    "Pop1" numeric,
    "Pop2" numeric,
    "Pop3" numeric,
    "Pop4" numeric,
    "Pop5" numeric,
    "Pop6" numeric,
    "Pop7" numeric,
    "Pop8" numeric,
    "Pop9" numeric,
    "Pop10" numeric,
    "Pop11" numeric,
    "Pop12" numeric,
    "Pop13" numeric,
    "Pop14" numeric,
    "Pop15" numeric,
    "Pop16" numeric,
    "Pop17" numeric,
    "Pop18" numeric,
    "Pop19" numeric,
    "Pop20" numeric,
    "Pop21" numeric,
    "Pop22" numeric,
    "Pop23" numeric,
    "Pop24" numeric,
    "Pop25" numeric,
    "Pop26" numeric,
    "Lb" integer
    ''',
    DemographicCountry = '''
    "Code" VARCHAR(3) PRIMARY KEY,
    "Name" VARCHAR(100)
    ''',
    DemographicLabels = '''
    "Indicator" VARCHAR(50) PRIMARY KEY,
    "Description" VARCHAR(100)
    ''',
    Demographic = '''
    "IndicatorId" VARCHAR(50),
    "CountryId" VARCHAR(3),
    "Year" integer,
    "Value" numeric,
    "Magnitude" VARCHAR(50),
    "Qualifier" VARCHAR(50)
    '''
)

In [5]:
table_equi = {
    "Documentation_21June2021.doc" : None,
    "list_ctry_yrs_21June2021.xlsx" : None,
    "country_codes" : ("CountryCodes", 0),
    "notes" : ("Notes", 0),
    "pop"   : ("Population", 0),
    "MortIcd7"  : ("MortICD", 7),
    "Morticd8"  : ("MortICD", 8),
    "Morticd9"  : ("MortICD", 9),
    "Morticd10_part1" : ("MortICD", "10_1"),
    "Morticd10_part2" : ("MortICD", "10_2"),
    "Morticd10_part3" : ("MortICD", "10_3"),
    "Morticd10_part4" : ("MortICD", "10_4"),
    "Morticd10_part5" : ("MortICD", "10_5"),
    "DEM_COUNTRY.csv": ("DemographicCountry", 0),
    "DEM_LABEL.csv": ("DemographicLabels", 0),
    "DEM_DATA_NATIONAL.csv": ("Demographic", 0)
}

In [6]:
def create_table(cursor, table_type, icd_rev, timestamp):
    table_name = f"formatted.{table_type}_{icd_rev}_{timestamp}"

    cursor.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (
        {table_spec[table_type]}
    );
    ''')
    
    return table_name

def load_csv(cursor, table_name, filename):
    with open(filename, 'r') as csvfile:
        cursor.copy_expert(f'''
            COPY {table_name}
            FROM STDIN
            DELIMITER ','
            CSV HEADER;
        ''', csvfile)

In [12]:
conn = psycopg2.connect(dbname="adsdb", user="adsdb", password="adsdb")

In [13]:
cur = conn.cursor()
cur.execute('''CREATE SCHEMA IF NOT EXISTS formatted''')

for meta in glob.glob(f"{folder_persistent}/extracted/*/metadata.json"):
    
    with open(meta, 'r') as f:
        metadata = json.load(f)

    folder_path = os.path.dirname(meta)
    folder_base = os.path.basename(folder_path)
    
    contents = glob.glob(f"{folder_path}/*")
    
    name_sha, _, timestamp = folder_base.rpartition("-")
    name, _, sha = name_sha.rpartition("-")
    version = sha[:4] + "_" + timestamp.partition(".")[0]

    for i in contents:
        filename = os.path.basename(i)
        if filename == "metadata.json":
            continue
    
        table = table_equi.get(filename)
    
        if table is not None:
            target_table = create_table(cur, table[0], table[1], version)
            load_csv(cur, target_table, i)

            print("LOAD", filename, "==>", target_table)
        else:
            print("SKIP", filename)
            
conn.commit()

SKIP list_ctry_yrs_21June2021.xlsx
LOAD country_codes ==> formatted.CountryCodes_0_8c41_1642852775
SKIP Documentation_21June2021.doc
LOAD notes ==> formatted.Notes_0_a67b_1642852775
LOAD pop ==> formatted.Population_0_3b8f_1642852775
LOAD MortIcd7 ==> formatted.MortICD_7_22b4_1642852775
LOAD Morticd8 ==> formatted.MortICD_8_9366_1642852775
LOAD Morticd9 ==> formatted.MortICD_9_71c9_1642852775
LOAD Morticd10_part1 ==> formatted.MortICD_10_1_f695_1642852776
LOAD Morticd10_part2 ==> formatted.MortICD_10_2_ffb1_1642852776
LOAD Morticd10_part3 ==> formatted.MortICD_10_3_1c6a_1642852776
LOAD Morticd10_part4 ==> formatted.MortICD_10_4_453e_1642852776
LOAD Morticd10_part5 ==> formatted.MortICD_10_5_8f17_1642852776
SKIP list_ctry_yrs_21June2021.xlsx
LOAD country_codes ==> formatted.CountryCodes_0_8c41_1642868814
SKIP Documentation_21June2021.doc
LOAD notes ==> formatted.Notes_0_a67b_1642868814
LOAD pop ==> formatted.Population_0_3b8f_1642868814
LOAD MortIcd7 ==> formatted.MortICD_7_22b4_1642868